In [1]:
from finta import TA
import yfinance as yf
import numpy as np
import pandas as pd
import yfinance as yf
from alpha_vantage.timeseries import TimeSeries
import datetime as dt
import copy
import matplotlib.pyplot as plt
import time
import json
from csv import writer
import pandas as pd
import time,datetime
from kiteconnect import KiteConnect,KiteTicker
from selenium import webdriver
import openpyxl
import pathlib

In [2]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="nu3ion96bs510ej9")

url=kite.login_url()


In [3]:
url

'https://kite.trade/connect/login?api_key=nu3ion96bs510ej9&v=3'

In [3]:
data = kite.generate_session("zYK3vA1QINKjWN4YSStoLbJY40A7nwRS", api_secret="1dp9cdmj7a8sr4pji0yy9oy6lkr00uun")
kite.set_access_token(data["access_token"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


In [ ]:
kite.instruments()

In [ ]:
kite.holdings()

In [5]:
df = pd.read_csv("zerodha.csv")

In [ ]:
df

In [6]:
traded_price={}
trading_symbol={}
tradable=[]



In [7]:
def trade_signal(instrument,l_s):
    
    global ltp,position,tickers,m,df,traded_price
    price=df['START TIME / PRICE'][k]


    signal=""
    
    if l_s=='BUY':


        instrument_token=instrument_tok(instrument)
        ltp=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])     
        
        
        if ltp>df['tradable/non-tradable'][m]+df['BUY ABOVE'][m]:
            signal='buy'

   
    if l_s=='SELL':


        instrument_token=instrument_tok(instrument)
        ltp=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])     
        
        
        if ltp<df['tradable/non-tradable'][m]-df['SELL BELOW'][m]:
            signal='sell'                       
      




    return signal   

In [8]:
tickers=[]

In [9]:
condition={}

In [10]:
instrument=kite.instruments()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 1114969


In [11]:
def position_now(instrument):
    global holding,tickers,df,k
    signal=""
    for i in range(len(holding)):
        for ticker in tickers:
            if holding[i]["tradingsymbol"]==df['TRADING SYMBOL'][k] and holding[i]["quantity"]==df["quantity"][k]:
                if holding[i]['quantity']>0:
                    signal= 'long'

                if holding[i]['quantity']<0:
                    signal= 'short'

    return signal

In [12]:
def instrument_tok(stock):

    global instrument
    for i in range(len(instrument)):
        if instrument[i]['tradingsymbol']==stock:
            return instrument[i]['instrument_token']


In [13]:
tradable=[] 



In [14]:
def to_be_traded(instrument):
    global ltp,position,tickers,k,df,traded_price,tradable,condition

    price=df['START TIME / PRICE'][k]

    if price-df['Tolerance Range'][k]<ltp<price+df['Tolerance Range'][k]:
        df['tradable/non-tradable'][k]='tradable'

        instrument_token=instrument_tok(instrument)
        df['become tradable at'][k]=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])


In [15]:
def square_off_signal(instrument):
    global ltp,tickers,holding,instrument_token,position,k,trading,tradable,condition,df,m,z
    instrument_token=instrument_tok(instrument)
    ltp=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])
    if df['FIRST Trade'][z]=='BUY':
        if ltp<df['stoploss'][z]:
            signal='squareoffsell'

        if ltp>df['stoploss'][z]:
            signal='squareoffbuy'
    
    return signal
    

In [16]:
def market_order1(instrument):
    global ltp,tickers,holding,instrument_token,position,k,trading,tradable,condition,df,m,z
    
    instrument_token=instrument_tok(instrument)
    df['ltp'][m]=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])
    df['stoploss'][z]=float(df['ltp'][m]+df['BUY ABOVE'][m])

In [17]:
def market_order(instrument):
    global ltp,tickers,holding,instrument_token,position,k,trading,tradable,condition,df,m,z
    
    instrument_token=instrument_tok(instrument)
    df['ltp'][m]=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])
    df['stoploss'][z]=float(df['ltp'][m]-df['SELL BELOW'][m])
    
    
    

In [18]:
def main():
    global ltp,tickers,holding,instrument_token,position,k,trading,tradable,condition,df,m,z,df1
    
    
    
    df = pd.read_csv("zerodha.csv")
    tickers=[]
    for i in range(len(df)):
        if df['on/off'][i]=='on' and df['tradable/non-tradable'][i]=='non-tradable' and df['order-ID'][i]==0:
            tickers.append(df['S.No.'][i])


    # holding=kite.holdings()
    
    for ticker in tickers:
        
        for i in range(len(df)):
            if df['S.No.'][i]==ticker:
                ticker=df['BASE FOR PRICE'][i]
                k=df['S.No.'][i]
                break
        # position=position_now(ticker)
        instrument_token=instrument_tok(ticker)
        ltp=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])
        to_be_traded(ticker)



  
        
        


    tradable=[]
    for i in range(len(df)):
        if df['tradable/non-tradable'][i]=='tradable' and df['on/off'][i]=='on' and df['order-ID'][i]==0:
            tradable.append(df['S.No.'][i])

    
    for ticker in tradable:
        
        for i in range(len(df)):
            if int(df['S.No.'][i])==int(ticker):
                ticker=df['TRADING SYMBOL'][i]
                m=df['S.No.'][i]
                break
                
        
        signal=trade_signal(ticker,df['FIRST Trade'][m])
        

        

        if 'buy' in signal:
            market_order(ticker)
    

        
            print("New long position initiated for ",ticker)

        if 'sell' in signal:
            market_order1(ticker)
            print("New short position initiated for ", ticker)
        

    trading=[]

    for i in range(len(df)):
        if df['order-ID'][i]!=0 and df['on/off'][i]=='on':
            trading.append(df['S.No.'][i])


    for ticker in trading:

        for i in range(len(df)):
            if int(df['S.No.'][i])==int(ticker):
                ticker=df['TRADING SYMBOL'][i]
                z=df['S.No.'][i]
                break
        
        ltp_previous=df['ltp'][z]
        stoploss_previous=df['stoploss'][z]
        df['ltp'][z]=float(kite.quote(str(instrument_token))[str(instrument_token)]['depth']['sell'][0]['price'])
        if df['FIRST Trade'][z]=='BUY':
            if df['ltp'][z]>ltp_previous:
                df['stoploss'][z]=stoploss_previous+abs(df['ltp'][z]-ltp_previous)

        
        if df['FIRST Trade'][z]=='SELL':
            if df['ltp'][z]<ltp_previous:
                df['stoploss'][z]=stoploss_previous-abs(df['ltp'][z]-ltp_previous)


        signal= square_off_signal(ticker)
        if 'squareoffbuy' in signal:

            market_order(ticker)



        if 'squareoffsell' in signal:
            market_order1(ticker)

    df1=df
    df.to_csv('zerodha.csv')



In [20]:
start_time=time.time()

timeout=start_time+60*60*5+45*60
times=time.time()

while time.time()<=timeout:

        

    main()
    print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))


    # except Exception as e:
    #     print(e)
        

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


passthrough at  2021-08-11 21:44:23


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


passthrough at  2021-08-11 21:44:24


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


passthrough at  2021-08-11 21:48:51


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


passthrough at  2021-08-11 21:48:52


ReadTimeout: HTTPSConnectionPool(host='api.kite.trade', port=443): Read timed out. (read timeout=7)